    Import libs

In [ ]:
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from collections import Counter

import pandas as pd
import numpy as np
import warnings
import math
import glob

warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
#increases the viewing size of the notebook to adjust to fit screen
#reduces line breaks and the amount of wasted space


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Ingest files

In [ ]:
#To do

# drop the occ_code per sheet

# drop rows that contain occ_group: detailed, minor

# drop occ_title 'All Occupations'

In [ ]:
# #take a look at the df to see how everything worked out
# data.head()

# #lots of NaN

# data.info()
# #need to convert a lot of columns

#data.shape

#(18593, 32)

In [ ]:
national_all = [] #appending all tables into one

for year in range(1999,2017+1): #python drops the last
    national_all.append(pd.read_csv('national_{}.csv'.format(year)))
    national_all[-1]['year'] = year #append year as the last column

In [ ]:
data = pd.concat(national_all)

In [ ]:
data.head()

clean up the dataframe

In [ ]:
# create a function to help clean up columns as needed
# some numeric columns are given in strings, they need to bes converted into floats
#mitigate empties (nan)
def convert_string_cols_into_float(col):
    
#     Initialize a list
    cleaned_col = []    
#     iterate over the column
    for i in col:        
#       check if there are any asterick patterns
        if i in ["***", "**","*"]:
            a = 0            
#       check if there are any nan values
        elif type(i) in [float,int] and math.isnan(i):
            a= 0
#       convert the string value into float
        elif type(i)==str:
#           remove the comma in between the string before converting them into float
            a = float(i.replace(",",""))
        cleaned_col.append(a)
        
    return cleaned_col

In [ ]:
# keep only used columns
# easy to amend as research expands (looking into the variances (10 vs 90 percentiles to ID the professions with high or low variance))
# ? would those professions be more geologically constrained and so the local job market forces the wages to be similiar? Will be interesting to find out
# once I locate employment and wage data that is geographically referenced 
data_clean = data[["occ_code","occ_title", "group", "tot_emp", "h_mean","a_mean", "year", "occ_group"]]

# group will be used to help identify macro trends since it comprises its components, its also important to remove it when looking at 
#individual professions since it could loop itself


In [ ]:
# convert string columns into float
data_clean.a_mean = convert_string_cols_into_float(data_clean.a_mean)
data_clean.h_mean = convert_string_cols_into_float(data_clean.h_mean)
data_clean.tot_emp = convert_string_cols_into_float(data_clean.tot_emp)

In [ ]:
# export datasets for backup purposes

data.to_csv('wage_data_backup.csv', index=False)
data_clean.to_csv('wage_data_clean_backup.csv', index=False)

In [ ]:
data.head()

In [ ]:
data_clean.head()

In [ ]:
# extract out all different types of jobs
all_group = set(data["occ_group"])

In [ ]:
all_group

In [ ]:
all_group1 = set(data["group"])

In [ ]:
all_group1

ingest the constant dollar values

In [ ]:
#load the csv

dollar = pd.read_csv('constant_dollar_jan_2018.csv')

In [ ]:
dollar.head()

In [ ]:
# join the dfs to assist in the calculations, likely a more efficient way of doing this

# constand_dollar_jan_2018

In [ ]:
constant_dollar_initial = pd.merge(data_clean, dollar, on='year', how='left')

In [ ]:
# take a view of the updated data

constant_dollar_initial.tail()

In [ ]:
# take a look at the dtypes

constant_dollar_initial.dtypes

# need to convert the object columns into the propr float

In [ ]:
# put the data into constant jan 2018 dollars by #col * rate

constant_dollar_initial['h_constant'] = constant_dollar_initial['h_mean']*constant_dollar_initial['rate']
constant_dollar_initial['a_constant'] = constant_dollar_initial['a_mean']*constant_dollar_initial['rate']

In [ ]:
# data review

# constant_dollar_initial.head()

Split the dataset into two: one for the groups and one for individual professions

In [ ]:
constant_dollar_initial.head()

In [ ]:
# create a subset for the groups, this will remove the possibility of the effectively dubplicate data messing with things
# it will also allow for the removal of the group column, further cleaning up the dataset for visualizations

# constant_group = constant_dollar_initial[constant_dollar_initial.group == 'major']

constant_ind = constant_dollar_initial[constant_dollar_initial.occ_group != 'major']

# 'broad', 'detailed', 'major', 'minor', nan, 'total'

In [ ]:
constant_ind = constant_ind[constant_ind.occ_group != 'detailed']
constant_ind = constant_ind[constant_ind.occ_group != 'minor']
constant_ind = constant_ind[constant_ind.occ_group != 'total']
constant_ind = constant_ind[constant_ind.occ_group != 'major']
constant_ind = constant_ind[constant_ind.group != 'major']
constant_ind = constant_ind[constant_ind.group != 'total']

In [ ]:
constant_ind.head()

In [ ]:
constant_ind = constant_ind.drop('occ_code', axis=1)


In [ ]:
constant_ind = constant_ind.drop('group', axis=1)


In [ ]:
constant_ind = constant_ind.drop('occ_group', axis=1)

In [ ]:
constant_ind

In [ ]:
# data review

#constant_group.head()
#constant_ind.head()

#looks gtg

In [ ]:
# # data cleaning
# #drop the group column, the occ_code is a simple way when manually looking at the data to see if it is correctly grouped
# #all the major groups are in the following format xx-0000
# # when creating the vizualizations, its easy to see which group is where

# constant_group.drop('group', axis=1)
# constant_ind.drop('group', axis=1)

Save wage data for visualizations

In [ ]:
# constant_group.to_csv('constant_group.csv', index=False)
constant_ind.to_csv('constant_ind.csv', index=False)

In [ ]:
#load the ind to fill in the errors
constant_ind = pd.read_csv('constant_ind.csv')


In [ ]:
constant_ind.head()

In [ ]:
constant_ind.shape

In [ ]:
# constant_ind_no_titlegroup = constant_ind.drop('occ_title', axis=1)
# constant_ind_no_titlegroup = constant_ind_no_titlegroup.drop('group', axis=1)

In [ ]:
# constant_ind_no_titlegroup['occ_code'] = constant_ind_no_titlegroup['occ_code'].str.replace('-','~')

In [ ]:
# constant_ind_no_titlegroup.to_csv('constant_ind_no_titlegroup.csv', index=False)

In [ ]:
# constant_ind_no_titlegroup.dtypes

Get the land value data sets corrected

    split the column that has year and quarter into two columns
        convert the quarter column into month

In [ ]:
land_states = pd.read_csv('landdata-states-2016q1.csv')
land_metro = pd.read_csv('landdata-msas-2016q1.csv')

In [ ]:
# #take a look at the datasets

land_metro.head()
# land_states.head()

In [ ]:
#clean up the column headers

land_states.columns = land_states.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
land_metro.columns = land_metro.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

In [ ]:
land_states.head()

In [ ]:
land_states['year'], land_states['quarter'] = zip(*land_states['date'].map(lambda x: x.split('Q')))
land_metro['year'], land_metro['quarter'] = zip(*land_metro['date'].map(lambda x: x.split('Q')))

In [ ]:
# take a look
#land_states.head()
#land_metro.head()

In [ ]:
land_states.shape

In [ ]:
land_metro.shape

In [ ]:
# combined data is 14,211 by 8 for 113,688

In [ ]:
# inspect

#land_states.dtypes
land_metro.dtypes

# state                object
# date                 object
# home_value           object
# structure_cost       object
# land_value           object
# land_share_pct       object
# home_price_index    float64
# land_price_index    float64
# year                 object
# quarter              object
# dtype: object

In [ ]:
#Convert the columns to the correct data types is required

#land_state

#good
#good
land_states['home_value'] = land_states['home_value'].apply(lambda x: x.replace('$', '').replace(',', '')).astype('float')
land_states['structure_cost'] = land_states['structure_cost'].apply(lambda x: x.replace('$', '').replace(',', '')).astype('float')
land_states['land_value'] = land_states['land_value'].apply(lambda x: x.replace('$', '').replace(',', '')).astype('float')
land_states['land_share_pct'] = land_states['land_share_pct'].apply(lambda x: x.replace('%', '')).astype('float') / 100
# good
# good
land_states['year'] = land_states['year'].apply(lambda x: x.replace('$', '').replace(',', '')).astype('int')
land_states['quarter'] = land_states['quarter'].apply(lambda x: x.replace('$', '').replace(',', '')).astype('int')

In [ ]:
#Convert the columns to the correct data types is required

#land_metro

#good
#good
land_metro['home_value'] = land_metro['home_value'].apply(lambda x: x.replace('$', '').replace(',', '')).astype('float')
land_metro['structure_cost'] = land_metro['structure_cost'].apply(lambda x: x.replace('$', '').replace(',', '')).astype('float')
land_metro['land_value'] = land_metro['land_value'].apply(lambda x: x.replace('$', '').replace(',', '')).astype('float')
land_metro['land_share_pct'] = land_metro['land_share_pct'].apply(lambda x: x.replace('%', '')).astype('float') / 100
# good
# good
land_metro['year'] = land_metro['year'].apply(lambda x: x.replace('$', '').replace(',', '')).astype('int')
land_metro['quarter'] = land_metro['quarter'].apply(lambda x: x.replace('$', '').replace(',', '')).astype('int')

In order to reflect the same scale as the employment data, dropping years prior to 1999

In [ ]:
land_states_constant = land_states[land_states.year > 1998]

In [ ]:
land_states_constant.head()

In [ ]:
land_metro_constant = land_metro[land_metro.year > 1998]

In [ ]:
land_metro_constant.head()

In [ ]:
# join with the constant dollar multiplier

land_states_constant = pd.merge(land_states_constant, dollar, on='year', how='left')

In [ ]:
land_metro_constant = pd.merge(land_metro_constant, dollar, on='year', how='left')

In [ ]:
#land_metro_constant.head()

In [ ]:
# create the current values

land_states_constant['home_value_constant'] = land_states_constant['home_value']*land_states_constant['rate']
land_states_constant['structure_cost_constant'] = land_states_constant['structure_cost']*land_states_constant['rate']
land_states_constant['land_value_constant'] = land_states_constant['land_value']*land_states_constant['rate']


In [ ]:
#land_states_constant.head()

In [ ]:
land_metro_constant['home_value_constant'] = land_metro_constant['home_value']*land_metro_constant['rate']
land_metro_constant['structure_cost_constant'] = land_metro_constant['structure_cost']*land_metro_constant['rate']
land_metro_constant['land_value_constant'] = land_metro_constant['land_value']*land_metro_constant['rate']

In [ ]:
#land_metro_constant.head()

In [ ]:
# export the data for visualizations
land_states_constant.to_csv('land_states_constant.csv', index=False)
land_metro_constant.to_csv('land_metro_constant.csv', index=False)

Corp Profit

In [ ]:
#load the csv

corp_profit = pd.read_csv('annual_pretax_corporate_profit.csv')

In [ ]:
constant_corp_profit = pd.merge(corp_profit, dollar, on='year', how='left')

In [ ]:
constant_corp_profit['profit_constant_billions'] = constant_corp_profit['corp_pretaxt_profit_billions']*constant_corp_profit['rate']

In [ ]:
#view

constant_corp_profit

In [ ]:
#save/export updated

constant_corp_profit.to_csv('constant_corp_profit.csv', index=False)

# Finished with data 

Do another backup of data/work
